# Science Wall

In [1]:
import urllib.request
import json
import pandas as pd
from token_key import k
import ssl
import re
import random
regex = re.compile("[А-ЯЁЙЁ]{3,}") 

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

## 0. Функция, читающая url

In [3]:
def readUrl(url, key=k):
    url = url + k
    content = urllib.request.urlopen(url, context=ctx)
    decoded_content = content.read().decode("utf-8")
    data = json.loads(decoded_content)
    return data

## 1. Собираем все id постов со страницы

In [4]:
def post_numbers(owner_id:str, offset_limit:int):
    data = readUrl("https://api.vk.com/method/wall.get?owner_id={}&count=100&v=5.95&access_token=".format(owner_id))
    post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
    offset = 101
    while offset < offset_limit:
        data = readUrl("https://api.vk.com/method/wall.get?owner_id={}&offset={}&count=100&v=5.95&access_token=".format(owner_id, offset))
        new_post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
        post_ids.update(new_post_ids)
        offset += 100
    return post_ids

In [5]:
# больше подписчиков
post_ids = post_numbers(owner_id="-29559271", 
                        offset_limit=40002)

# https://vk.com/sci

In [16]:
len(post_ids)

40100

In [19]:
post_ids = {k:v for k,v in post_ids.items() if k not in df_res.parent_id.values.tolist()}

In [20]:
len(post_ids)

39213

## 2. Собираем все комментарии с капслоком на русском языке по пост-id

In [21]:
def get_comments(owner_id:str,  
                 post_ids:list,
                offset_limit=1002):

    result_entries = []
    for id_ in post_ids:
        offset = 1
        while offset < offset_limit: 
            data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&post_id={}&offset={}&count=100&v=6.7&access_token=".format(owner_id, str(id_), offset))
            try:
                data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                data_list = data.values.tolist()
                lines = [i[2] for i in data_list]
                offset += 100
                for i, line in enumerate(lines):
                    try:
                        if re.search(regex, line):
                            result_entries.append(data_list[i])
                    except TypeError:
                        continue
                
                comment_ids = [i[3] for i in data_list]
                for com_id in comment_ids:
                    comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&comment_id={}&count=100&v=6.7&access_token=".format(owner_id, com_id))
                    comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                    comment_data_list = comment_data.values.tolist()
                    lines2 = [i[2] for i in comment_data_list]
                    for i, line in enumerate(lines2):
                        try:
                            if re.search(regex, line):
                                result_entries.append(comment_data_list[i])
                        except TypeError:
                            pass
                        
                if offset%100 == 1:
                    print(offset, "ids parsed")

            except KeyError:
                continue
                
    return result_entries

In [22]:
#capslock_comments = get_comments(owner_id="-145800770",
#                                 post_ids=post_ids,
#                                 offset_limit=1002)

In [28]:
offset_limit = 1001
owner_id = "-29559271"

result_entries = []
count = 0
count_ids = 0
for id_, text in post_ids.items():
    offset = 1
    while offset < offset_limit: 
        data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&post_id={}&offset={}&count=100&v=6.7&access_token=".format(owner_id, id_, offset))
        try:
            data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
            data_list = data.values.tolist()
            for i, lst in enumerate(data_list):
                data_list[i].append(text)
            lines = [i[2] for i in data_list]
            offset += 100
            for i, line in enumerate(lines):
                try:
                    if re.search(regex, line):
                        result_entries.append(data_list[i])
                        count += 1
                        if count%100 == 0:
                            print(count, "comments with capslock collected")
                except TypeError:
                    continue
                
            comment_ids_and_text = {i[3]:i[2] for i in data_list}
            for com_id, com_text in comment_ids_and_text.items():
                comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&comment_id={}&count=100&v=6.7&access_token=".format(owner_id, com_id))
                comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "parents_stack", "text", "id"])
                comment_data_list = comment_data.values.tolist()
                for i, lst in enumerate(comment_data_list):
                    comment_data_list[i].append(com_text)
                lines2 = [i[2] for i in comment_data_list]
                for i, line in enumerate(lines2):
                    try:
                        if re.search(regex, line):
                            result_entries.append(comment_data_list[i])
                            count += 1
                            if count%100 == 0:
                                print(count, "comments with capslock collected")
                    except TypeError:
                        pass
        except KeyError:
            continue
    count_ids += 1
    if count_ids % 50 == 0:
        print(count_ids, "ids proceeded")

50 ids proceeded
100 ids proceeded
100 comments with capslock collected
150 ids proceeded
200 ids proceeded
200 comments with capslock collected
300 comments with capslock collected
250 ids proceeded
400 comments with capslock collected
500 comments with capslock collected
600 comments with capslock collected
700 comments with capslock collected
800 comments with capslock collected
300 ids proceeded
900 comments with capslock collected
1000 comments with capslock collected
1100 comments with capslock collected
1200 comments with capslock collected
1300 comments with capslock collected
350 ids proceeded
1400 comments with capslock collected
1500 comments with capslock collected
1600 comments with capslock collected
1700 comments with capslock collected
1800 comments with capslock collected
1900 comments with capslock collected
400 ids proceeded
2000 comments with capslock collected
2100 comments with capslock collected
2200 comments with capslock collected
2300 comments with capslock co

KeyboardInterrupt: 

In [29]:
df_res = pd.DataFrame(result_entries, columns = ["from_id", "parent_id", "comment_text", "comment_id", "parent_text"])
df_res.count()


from_id         8742
parent_id       8742
comment_text    8742
comment_id      8742
parent_text     8601
dtype: int64

In [39]:
df_res_1 = pd.read_csv("t.csv")
df_res_1.drop(columns = ["Unnamed: 0"], inplace=True,axis=1)
df_res_1.head()

,from_id,parent_id,comment_text,comment_id,parent_text
0,2374206.0,1690025,В США даже полиция не имеет права войти в част...,1690057,NaN
1,175885079.0,1690025,"В США не посадят, а у нас легко, ты вообще не ...",1690276,NaN
2,246735317.0,1690025,"Почитал комменты, в основном люди правильно оц...",1690332,NaN
3,87059047.0,1690025,МАЛАДЦА !!!,1690403,NaN
4,80183423.0,1690025,Правильно сделала совершенно! На тачку то напл...,1690445,NaN


In [41]:
len(df_res_1)

11887

In [42]:
df_res.head()

,comment_id,comment_text,from_id,index,parent_id,parent_text
0,1690057,В США даже полиция не имеет права войти в част...,2374206.0,NaN,1690025,NaN
1,1690276,"В США не посадят, а у нас легко, ты вообще не ...",175885079.0,NaN,1690025,NaN
2,1690332,"Почитал комменты, в основном люди правильно оц...",246735317.0,NaN,1690025,NaN
3,1690403,МАЛАДЦА !!!,87059047.0,NaN,1690025,NaN
4,1690445,Правильно сделала совершенно! На тачку то напл...,80183423.0,NaN,1690025,NaN


In [35]:
new_df_res = pd.concat([df_res_1, df_res])
df_res = new_df_res.reset_index(drop=True)
df_res.head()

f:\ann\program files\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,comment_id,comment_text,from_id,index,parent_id,parent_text
0,1690057,В США даже полиция не имеет права войти в част...,2374206.0,NaN,1690025,NaN
1,1690276,"В США не посадят, а у нас легко, ты вообще не ...",175885079.0,NaN,1690025,NaN
2,1690332,"Почитал комменты, в основном люди правильно оц...",246735317.0,NaN,1690025,NaN
3,1690403,МАЛАДЦА !!!,87059047.0,NaN,1690025,NaN
4,1690445,Правильно сделала совершенно! На тачку то напл...,80183423.0,NaN,1690025,NaN


In [45]:
df_res.drop(columns=["index"], inplace=True)
df_res.head(30)

,comment_id,comment_text,from_id,parent_id,parent_text
0,1690057,В США даже полиция не имеет права войти в част...,2374206.0,1690025,NaN
1,1690276,"В США не посадят, а у нас легко, ты вообще не ...",175885079.0,1690025,NaN
2,1690332,"Почитал комменты, в основном люди правильно оц...",246735317.0,1690025,NaN
3,1690403,МАЛАДЦА !!!,87059047.0,1690025,NaN
4,1690445,Правильно сделала совершенно! На тачку то напл...,80183423.0,1690025,NaN
5,1690174,"[id209103798|Дима], условия жизни и зп сравни....",141006599.0,[1690029],И ей не предъявили обвинений получается за пре...
6,1690181,сколько человек в США за чертой бедности? А в ...,5657313.0,[1690029],И ей не предъявили обвинений получается за пре...
7,1690208,"[id136175042|Ksenia], это вы про какую заграни...",1927460.0,[1690029],И ей не предъявили обвинений получается за пре...
8,1690345,Иная ситуация. Вы владелец фирмы с прибылью 10...,5657313.0,[1690029],И ей не предъявили обвинений получается за пре...
9,1690260,"[id332820940|Евгений], а она рядом с вами и н...",937113.0,[1690056],"Убийца. Пожизненное ей, такие люди не должны х..."


In [46]:
len(df_res)

32516

## 4. Извлекаем инфу о каждом комментаторе и добавляем в датафрейм. 

In [47]:
def chunks(list_of_values, chunk_size=400):
    return [list_of_values[i:i + chunk_size] for i in range(0, len(list_of_values), chunk_size)]

In [48]:
def update_df_with_user_info(df):
    
    df["first_name"] = None 
    df["last_name"] = None 
    df["sex"] = None
    df["bdate"] = None 
    df["city"] = None
    
    df.drop_duplicates(subset=["from_id"], keep='first', inplace=True)
    df["from_id"] = df["from_id"].apply(lambda x: int(x))
    from_ids = [str(df.from_id[i]) for i in df.index]
    df = df.set_index("from_id")
        
    all_data = []
    for ids in chunks(from_ids):
        k = ",".join(ids)
        data = readUrl("https://api.vk.com/method/users.get?user_ids={}&fields=bdate,sex,city&v=6.7&access_token=".format(str(k)))
        all_data += data["response"]
    
    print(all_data[0])
    
    for el in all_data:
        indx = el["id"]
        try:
            el["city"] = el["city"]["title"]
        except KeyError:
            continue
        df.loc[indx, "first_name"] = el.get("first_name", None)
        df.loc[indx, "last_name"] = el.get("last_name", None)
        df.loc[indx, "sex"] = el.get("sex", None)
        df.loc[indx,"bdate"] = el.get("bdate", None)
        df.loc[indx,"city"] = el.get("city", None)
        
    return df

In [49]:
df_res = update_df_with_user_info(df_res)

{'id': 2374206, 'first_name': 'Andrey', 'last_name': 'Kulkov', 'is_closed': True, 'can_access_closed': False, 'sex': 2, 'bdate': '15.7.1987', 'city': {'id': 910, 'title': 'Egoryevsk'}}


In [50]:
df_res.tail()

,comment_id,comment_text,parent_id,parent_text,first_name,last_name,sex,bdate,city
from_id,,,,,,,,,
64838301,875389,"«Приставы не смогли взыскать в прошлом году 2,...",875268,,Dima,Dudukin,2,None,Ulyanovsk
27756845,875451,Только гражданам ))А иностранцам гражданство О...,875268,,None,None,None,None,None
10722108,876004,А у нас переписали определение БЕДНОСТЬ.,875268,,Nikolay,Balandin,2,21.10.1985,Vologda
180573259,877262,"Есть ещё одно НО, это Россия выделит $100 млн ...",875268,,Petr,Potekhin,2,13.1.1994,Saint Petersburg
6614285,877188,"[id289068065|Ink], в США?",[875600],Живём в ублюдочной стране.,Vladimir,Zharavin,2,None,Kandalaksha


## 5. Удалим комменты с пустыми полями sex, bdate, city

In [51]:
df_test = df_res.dropna(subset=["sex", "bdate", "city"],how="all") 

In [52]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9030 entries, 2374206 to 6614285
Data columns (total 9 columns):
comment_id      9030 non-null int64
comment_text    9030 non-null object
parent_id       9030 non-null object
parent_text     6575 non-null object
first_name      9030 non-null object
last_name       9030 non-null object
sex             9030 non-null object
bdate           7003 non-null object
city            9030 non-null object
dtypes: int64(1), object(8)
memory usage: 423.3+ KB


In [53]:
df_test.sex.value_counts()
# со 100 постов собралось всего 287 комментариев с капслоком 

2    6871
1    2159
Name: sex, dtype: int64

In [54]:
df_test.city.value_counts()

Moscow              1418
Saint Petersburg    1097
Yekaterinburg        180
Novosibirsk          144
Krasnodar            139
Samara               132
Chelyabinsk          128
Rostov-on-Don        127
Krasnoyarsk          115
Perm                 111
Minsk                107
Kazan                104
Volgograd             92
Voronezh              91
Nizhny Novgorod       88
Ufa                   86
Tyumen                72
Omsk                  69
Kaliningrad           63
Almaty                62
Tolyatti              61
Saratov               60
Vladivostok           54
Irkutsk               54
Sevastopol            53
Donetsk               50
Orenburg              50
Murmansk              50
Kyiv                  46
Tomsk                 44
                    ... 
Torez                  1
Verona                 1
Volokolamsk            1
Novorzhev              1
Karshi                 1
Chausy                 1
Lenino                 1
Nantes                 1
San Juan               1


In [55]:
df_test.head()

,comment_id,comment_text,parent_id,parent_text,first_name,last_name,sex,bdate,city
from_id,,,,,,,,,
2374206,1690057,В США даже полиция не имеет права войти в част...,1690025,NaN,Andrey,Kulkov,2,15.7.1987,Egoryevsk
175885079,1690276,"В США не посадят, а у нас легко, ты вообще не ...",1690025,NaN,Vlad,Goncharenko,2,15.2,Saint Petersburg
87059047,1690403,МАЛАДЦА !!!,1690025,NaN,Alexander,Berezansky,2,12.10.1976,Moscow
80183423,1690445,Правильно сделала совершенно! На тачку то напл...,1690025,NaN,Veronika,Zemlyanika,1,10.1,Saint Petersburg
141006599,1690174,"[id209103798|Дима], условия жизни и зп сравни....",[1690029],И ей не предъявили обвинений получается за пре...,Alexey,Fomin,2,8.9.1990,Yaroslavl


In [56]:
df_test = df_test.reset_index()

In [58]:
df_test = df_test.drop_duplicates(subset=["from_id"])
df_test.count()

from_id         9030
comment_id      9030
comment_text    9030
parent_id       9030
parent_text     6575
first_name      9030
last_name       9030
sex             9030
bdate           7003
city            9030
dtype: int64

## Пишем функцию, которая собирает инфу о группах юзера

In [59]:
def update_df_with_subskr_info(df, offset_limit=1001):
    
    df = df.set_index("from_id")
    df["group_info"] = None
    indices = list(df.index)
        
    count = 0
    
    for i in indices: 
        offset = 0
        group_info = []
        while offset < offset_limit:
            subskr = readUrl("https://api.vk.com/method/users.getSubscriptions?user_id={}&extended=1&fields=activity,status,description&count=200&offset={}&v=6.7&access_token=".format(str(i), offset))
            offset += 200
            try:
                for j in subskr["response"]["items"]:
                    group = [j["name"], j["status"], j["description"], j["activity"]]
                    group_info.append(group)
            except KeyError:
                continue
            
        if group_info != []:
            group_info = [" -||- ".join(i) for i in group_info]
            group_info = " -|-|- ".join(group_info)
            df.loc[i, "group_info"] = group_info
        
        count += 1
        if count % 100 == 0:
            print(count, "users processed")
        
    return df

In [60]:
df = update_df_with_subskr_info(df_test)

100 users processed
200 users processed
300 users processed
400 users processed
500 users processed
600 users processed
700 users processed
800 users processed
900 users processed
1000 users processed
1100 users processed
1200 users processed
1300 users processed
1400 users processed
1500 users processed
1600 users processed
1700 users processed
1800 users processed
1900 users processed
2000 users processed
2100 users processed
2200 users processed
2300 users processed
2400 users processed
2500 users processed
2600 users processed
2700 users processed
2800 users processed
2900 users processed
3000 users processed
3100 users processed
3200 users processed
3300 users processed
3400 users processed
3500 users processed
3600 users processed
3700 users processed
3800 users processed
3900 users processed
4000 users processed
4100 users processed
4200 users processed
4300 users processed
4400 users processed
4500 users processed
4600 users processed
4700 users processed
4800 users processed
4

In [27]:
df.tail()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info
from_id,,,,,,,,,,
259335134,6711754,"[id211787099|Антонина], именно. Но, к сожалени...",6712100,,Anastasia,Uryutina,1,3.1.2001,Novouralsk,станция №12 -||- всё будет хорошо. -||- -||- ...
49847737,6711754,"[id68582541|Данил], я считаю, что всех чебурек...",6712471,,Grigory,Savenkov,2,18.6.1987,Luanda,None
217998036,6711754,Нужно любить своих детей. ЛЮБИТЬ.,6712676,,Galina,Yudina,1,2.12,Tolyatti,РАБОЧИЕ АВТОВАЗА -||- ПРОФСОЮЗ ЕДИНСТВО ТОЛЬЯТ...
330060237,6711556,"Дааа. "" У меня есть дырка между ног! Да, все с...",6711952,,Deniel,Semenov,2,None,Baley,None
105574664,6711556,"Как жаль, что раньше это была группа ПРО яжмат...",6712341,,Rita,Gusarova,1,8.6.1997,Saint Petersburg,"Onmyoji ASK -||- -||- Спасибо каждому, кто по..."


In [32]:
df["source"] = ["sci" for i in range(len(df))]

In [33]:
df.head()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
from_id,,,,,,,,,,,
6343530,1.17367e+07,Но это был еще не конец. Я только 31 декабря 1...,11736787,"20 лет назад, 9 августа 1999 года, президент Р...",Yulia,Vershinina,1,24.9,Saint Petersburg,None,lentach
135863776,1.17367e+07,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,11736862,"20 лет назад, 9 августа 1999 года, президент Р...",Alexey,Krutikov,2,31.8,Novosibirsk,PidorasBlog -||- паблик чечни -||- Магнит гомо...,lentach
10862032,1.17367e+07,Первый срок (и часть второго даже) я одобрял е...,11736889,"20 лет назад, 9 августа 1999 года, президент Р...",Maxim,Gerasimov,2,5.8,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach
29670766,[11736731],"[id164270524|Костик], я думаю много весёлого б...",11736884,Зря боря подарил стране плешивого фашиста .,Alexey,Kovalenko,2,18.6.1998,Saint Petersburg,None,lentach
320342320,[11736787],"[id6343530|Юлия], в какой из бывших республик ...",11737428,Но это был еще не конец. Я только 31 декабря 1...,Pavel,Vladimirovich,2,26.7,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach


In [34]:
df.reset_index(inplace=True)

In [35]:
df.head()

,from_id,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
0,6343530,1.17367e+07,Но это был еще не конец. Я только 31 декабря 1...,11736787,"20 лет назад, 9 августа 1999 года, президент Р...",Yulia,Vershinina,1,24.9,Saint Petersburg,None,lentach
1,135863776,1.17367e+07,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,11736862,"20 лет назад, 9 августа 1999 года, президент Р...",Alexey,Krutikov,2,31.8,Novosibirsk,PidorasBlog -||- паблик чечни -||- Магнит гомо...,lentach
2,10862032,1.17367e+07,Первый срок (и часть второго даже) я одобрял е...,11736889,"20 лет назад, 9 августа 1999 года, президент Р...",Maxim,Gerasimov,2,5.8,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach
3,29670766,[11736731],"[id164270524|Костик], я думаю много весёлого б...",11736884,Зря боря подарил стране плешивого фашиста .,Alexey,Kovalenko,2,18.6.1998,Saint Petersburg,None,lentach
4,320342320,[11736787],"[id6343530|Юлия], в какой из бывших республик ...",11737428,Но это был еще не конец. Я только 31 декабря 1...,Pavel,Vladimirovich,2,26.7,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8741 entries, 0 to 8740
Data columns (total 12 columns):
from_id         8741 non-null int64
parent_id       8741 non-null object
comment_text    8741 non-null object
comment_id      8741 non-null int64
parent_text     8542 non-null object
first_name      8741 non-null object
last_name       8741 non-null object
sex             8741 non-null object
bdate           6875 non-null object
city            8741 non-null object
group_info      6617 non-null object
source          8741 non-null object
dtypes: int64(2), object(10)
memory usage: 478.1+ KB


## 6. Сохраняем все в файл csv

In [37]:
df.to_csv("capitalization-lentach-wall.csv", index=False)